In [1]:
# Import Libraries
import pandas as pd
import pickle as pkl

In [2]:
# Define a dataframe for predictions

predXDict = {'age': [32],
             'sex': ['female'],
             'bmi': [21.3],
             'children': [2],
             'smoker': ['yes'],
             'region': ['southwest'],
             'bmi_zscore': [0.33]}

predXDF = pd.DataFrame.from_dict(predXDict)

predXDF.head()

,age,sex,bmi,children,smoker,region,bmi_zscore
0,32,female,21.3,2,yes,southwest,0.33


In [3]:
with open("nnModel.pkl", 'rb') as f:
    nnModel = pkl.load(f)

In [4]:
with open("encoder.pkl", 'rb') as f:
    encoder = pkl.load(f)

encoder.categories_

[array(['female', 'male'], dtype=object),
 array(['no', 'yes'], dtype=object),
 array(['northeast', 'northwest', 'southeast', 'southwest'], dtype=object)]

In [5]:
encodedData = encoder.transform(predXDF[['sex', 'smoker', 'region']])

encodedData

array([[1., 0., 0., 1., 0., 0., 0., 1.]])

In [6]:
column_names = [value for values in encoder.categories_ for value in values]

oneHotDF = pd.DataFrame(encodedData, columns=column_names)

oneHotDF.head()

,female,male,no,yes,northeast,northwest,southeast,southwest
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [7]:
oneHotDF.columns = oneHotDF.columns.get_level_values(0)

predXDF = pd.concat([predXDF, oneHotDF], axis=1).drop(columns=['sex', 'smoker', 'region'])

columns_except_bmi_zscore = [col for col in predXDF.columns if col != 'bmi_zscore']

# Now append 'bmi_zscore' to the end of this list
ordered_columns = columns_except_bmi_zscore + ['bmi_zscore']

# Reorder predXDF columns based on the list above
predXDF = predXDF[ordered_columns]

predXDF.head()

,age,bmi,children,female,male,no,yes,northeast,northwest,southeast,southwest,bmi_zscore
0,32,21.3,2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.33


In [8]:
with open("xScaler.pkl", 'rb') as f:
    xScaler = pkl.load(f)

with open("yScaler.pkl", 'rb') as f:
    yScaler = pkl.load(f)

In [9]:
colNames = predXDF.columns.values.tolist()

colNames

# use the scaler to transform the prediction data
predXDF[colNames] = xScaler.transform(predXDF[colNames])

predXDF.head()

,age,bmi,children,female,male,no,yes,northeast,northwest,southeast,southwest,bmi_zscore
0,-0.525181,-1.541925,0.774466,1.040476,-1.040476,-2.088557,2.088557,-0.576938,-0.573637,-0.586839,1.748291,0.350476


In [10]:
with open("linearModel.pkl", 'rb') as f:
    linearModel = pkl.load(f)

with open("SVMModel.pkl", 'rb') as f:
    svmModel = pkl.load(f)

with open("treeModel.pkl", 'rb') as f:
    treeModel = pkl.load(f)

with open("nnModel.pkl", 'rb') as f:
    nnModel = pkl.load(f)

In [11]:
linPred = linearModel.predict(predXDF)
svmPred = svmModel.predict(predXDF)
treePred = treeModel.predict(predXDF)
nnPred = nnModel.predict(predXDF)

linPred = yScaler.inverse_transform(linPred.reshape(-1, 1))
svmPred = yScaler.inverse_transform(svmPred.reshape(-1, 1))
treePred = yScaler.inverse_transform(treePred.reshape(-1, 1))
nnPred = yScaler.inverse_transform(nnPred.reshape(-1, 1))

print(f"linear prediction: {linPred}")
print(f"svm prediction: {svmPred}")
print(f"tree prediction: {treePred}")
print(f"neural net prediction: {nnPred}")

linear prediction: [[29075.05201055]]
svm prediction: [[21845.87792449]]
tree prediction: [[38998.546]]
neural net prediction: [[26295.91876224]]
